In [486]:
import pandas as pd
import numpy as np

In [487]:
df = pd.read_csv('df_with_markup.csv')

In [488]:
df.columns

Index(['pmcs', 'patient_description', 'Clinical characteristics',
       'C-reactive protein, mg/dL', 'D-Dimer', 'Age', 'Prothrombin time',
       'Sex', 'Creatinine mg/dl', 'White blood cell count', 'Death',
       'Comorbidities', 'Ferritin', 'Lactate dehydrogenase', 'Hemoglobin',
       'Platelet', 'COVID-19 medication', 'Lymphocytes', 'Fibrinogen',
       'Neutrophil', 'Alanine aminotransferase', 'Aspartate aminotransferase',
       'Albumin', 'BMI', 'Mechanical Ventilator Status', 'Creatine kinase',
       'Total bilirubin', 'Procalcitonin (ng/mL)', 'Oxygen saturation',
       'International normalised ratio (INR)', 'IL-6',
       'Erythrocyte sedimentation rate (ESR)', 'COVID-19 disease severity',
       'Lactate', 'Laboratory findings', 'Troponin I', 'MRI Findings',
       'Therapy', 'Age - year/ Sex', 'Oxygen therapy', 'NIHSS', 'Smoking',
       'Hematocrit (%)', 'ICU admission', 'Race', 'Hospital LOS (d)',
       'Highest temperature, °C', 'Monocytes', 'GFR (ml/min)',
       '

In [489]:
df_for_modelling = df.loc[:, ~df.columns.isin(['markup', 'ners_true'])]

In [490]:
type(df_for_modelling.iloc[0]['Prothrombin time'])

float

In [491]:
df_for_modelling.fillna('NaN', inplace=True)

/home/jovyan/.imgenv-gpu-a100-0/lib/python3.7/site-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [492]:
col_list_X = list(df_for_modelling.columns)
col_list_X.remove('Death')
col_list_X.remove('pmcs')
col_list_X.remove('patient_description')

In [493]:
df_for_modelling['patient_id'] = 'patient' + df_for_modelling.index.astype(str)

/home/user/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [494]:
df_triplets = pd.DataFrame(columns=['subject', 'predicate', 'object'])

for i, row in df_for_modelling.iterrows():
    pmc = row['patient_id']
    
    for col in col_list_X:
        if row[col] != 'NaN':
            if col == 'Clinical characteristics':
                clinical = row[col].split('; ')
                for c in clinical:
                    triplet = {'subject': pmc, 'predicate': col, 'object': c}
                    df_triplets = df_triplets.append(triplet, ignore_index=True)
            else:
                triplet = {'subject': pmc, 'predicate': col, 'object': row[col]}
                df_triplets = df_triplets.append(triplet, ignore_index=True)

In [568]:
len(df_triplets)

4269

In [13]:
df_triplets.to_csv('all_triplets_sep.tsv', sep='\t', header=False, index=False)

In [495]:
import torch
import pandas as pd
from pykeen.models import predict
from pykeen.pipeline import pipeline
from pykeen.triples import TriplesFactory
from pykeen.stoppers.early_stopping import EarlyStopper
from sklearn.model_selection import train_test_split
from tqdm import tqdm 
import numpy as np
import json

In [506]:
tf = TriplesFactory.from_path('all_triplets_sep.tsv')
training, testing = tf.split()

INFO:pykeen.triples.splitting:done splitting triples to groups of sizes [661, 850]


In [540]:
results_mure = pipeline(
    training=training,
    testing=testing,
    model='MuRE',
    training_loop='sLCWA',
    negative_sampler='basic',
    stopper='early',
    validation=testing,
    training_kwargs = dict(num_epochs=500),
    model_kwargs = dict(embedding_dim=300,
                        entity_bias_initializer=torch.nn.init.uniform_
                       ),
    stopper_kwargs=dict(frequency=3, patience=5, relative_delta=0.0001),
#     negative_sampler_kwargs=dict(
#         filtered=True,
#         filterer='python-set',  
#         num_negs_per_pos = 3, 
#         corruption_scheme=(0)
#     ),
    loss='softplus',
    random_seed=42,
    device='cuda:0'
)

INFO:pykeen.pipeline.api:Using device: cuda:0
/home/jovyan/.imgenv-gpu-a100-0/lib/python3.7/site-packages/pykeen/nn/representation.py:372: UserWarning: Directly use Embedding.max_id instead of num_embeddings.
  warnings.warn(f"Directly use {self.__class__.__name__}.max_id instead of num_embeddings.")


Training epochs on cuda:0:   0%|          | 0/500 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0/14 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/14 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/14 [00:00<?, ?batch/s]

...

Training batches on cuda:0:   0%|          | 0/14 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/14 [00:00<?, ?batch/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 0.08s seconds
INFO:pykeen.stoppers.early_stopping:Stopping early at epoch 243. The best result 0.35 occurred at epoch 228.


Evaluating on cuda:0:   0%|          | 0.00/850 [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 0.10s seconds


In [505]:
results_pairre = pipeline(
    training=training,
    testing=testing,
    model='PairRE',
    training_loop='sLCWA',
    negative_sampler='basic',
    stopper='early',
    validation=testing,
    training_kwargs = dict(num_epochs=300),
    model_kwargs = dict(embedding_dim=315, power_norm=True, p=2),
    negative_sampler_kwargs=dict(
        filtered=True,
        filterer='python-set',  
        num_negs_per_pos = 3, 
        corruption_scheme=(0)
    ),
    optimizer='adamax',
    random_seed=42,
    device='cuda:0'
)

INFO:pykeen.pipeline.api:Using device: cuda:0
INFO:pykeen.triples.triples_factory:Creating inverse triples.


Training epochs on cuda:0:   0%|          | 0/300 [00:00<?, ?epoch/s]

INFO:pykeen.triples.triples_factory:Creating inverse triples.


Training batches on cuda:0:   0%|          | 0/27 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/27 [00:00<?, ?batch/s]

...

Training batches on cuda:0:   0%|          | 0/27 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/27 [00:00<?, ?batch/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 0.08s seconds
INFO:pykeen.stoppers.early_stopping:Stopping early at epoch 30. The best result 0.1888235294117647 occurred at epoch 10.


Evaluating on cuda:0:   0%|          | 0.00/850 [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 0.10s seconds


In [541]:
patient_ids = df_for_modelling['patient_id'].to_list()
entity_embedding = results_mure.model.entity_representations[0]

patient_embeds = np.zeros((len(df_for_modelling['patient_id'].to_list()), 300))

for i, p_id in enumerate(patient_ids):
    try:
        id_t = entity_embedding(torch.as_tensor(training.entities_to_ids([p_id]), device='cuda')).cpu().detach().numpy()
        patient_embeds[i] = id_t[0]
    except:
        print(p_id)
        

In [542]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression, SGDClassifier

In [543]:
X_train, X_test, y_train, y_test = train_test_split(
    patient_embeds, 
    df_for_modelling['Death'], 
    test_size=0.3, 
    random_state=42
)

In [564]:
logreg = LogisticRegression(C=4, penalty='elasticnet', solver='saga', l1_ratio=0.1, max_iter=500)

In [565]:
# sgd = SGDClassifier(loss='squared_hinge', alpha=1)

In [566]:
logreg.fit(X_train, y_train)
# sgd.fit(X_train, y_train)

LogisticRegression(C=4, l1_ratio=0.1, max_iter=500, penalty='elasticnet',
                   solver='saga')

In [567]:
preds = logreg.predict(X_test)

print(classification_report(y_true=y_test, y_pred=preds))

              precision    recall  f1-score   support

           0       0.83      0.90      0.86        60
           1       0.40      0.27      0.32        15

    accuracy                           0.77        75
   macro avg       0.62      0.58      0.59        75
weighted avg       0.74      0.77      0.76        75



In [484]:
# logreg = LogisticRegression(C=0.75), emb_dim = 300

preds = logreg.predict(X_test)

print(classification_report(y_true=y_test, y_pred=preds))

              precision    recall  f1-score   support

           0       0.89      0.92      0.90        60
           1       0.62      0.53      0.57        15

    accuracy                           0.84        75
   macro avg       0.75      0.72      0.74        75
weighted avg       0.83      0.84      0.84        75



In [485]:
report = classification_report(y_test, preds, output_dict=True)
df_classification_report = pd.DataFrame(report).transpose()
df_classification_report = df_classification_report.round(decimals=2)

df_classification_report = df_classification_report.astype(str)
df_classification_report.at['accuracy', 'precision'] = ''
df_classification_report.at['accuracy', 'recall'] = ''
df_classification_report.at['accuracy', 'support'] = df_classification_report.at['macro avg', 'support']
df_classification_report['support'] = df_classification_report['support'].str.replace('[.]0', '')
print(df_classification_report.to_latex())

\begin{tabular}{lllll}
\toprule
{} & precision & recall & f1-score & support \\
\midrule
0            &      0.89 &   0.92 &      0.9 &      60 \\
1            &      0.62 &   0.53 &     0.57 &      15 \\
accuracy     &           &        &     0.84 &      75 \\
macro avg    &      0.75 &   0.72 &     0.74 &      75 \\
weighted avg &      0.83 &   0.84 &     0.84 &      75 \\
\bottomrule
\end{tabular}



In [52]:
# logreg = LogisticRegression(C=3), emb_dim = 300, corruption_scheme

preds = logreg.predict(X_test)

print(classification_report(y_true=y_test, y_pred=preds))

              precision    recall  f1-score   support

           0       0.87      0.87      0.87        60
           1       0.47      0.47      0.47        15

    accuracy                           0.79        75
   macro avg       0.67      0.67      0.67        75
weighted avg       0.79      0.79      0.79        75

